## Observations and Insights

## Dependencies and starter code

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata = "Data/Mouse_metadata.csv"
study_results = "Data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata)
study_results = pd.read_csv(study_results)

# Combine the data into a single dataset
combined_df = pd.merge(mouse_metadata, study_results, on="Mouse ID", how="left")

combined_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


## Summary statistics
* Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

In [2]:
sum_table = pd.DataFrame(combined_df.groupby("Drug Regimen").count())

sum_table["Mean Tumor Volume"] = pd.DataFrame(combined_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].mean())
sum_table["Median Tumor Volume"] = pd.DataFrame(combined_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].median())
sum_table["Tumor Volume Variance"] = pd.DataFrame(combined_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].var())
sum_table["Tumor Volume Std. Dev."] = pd.DataFrame(combined_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].std())
sum_table["Tumor Volume Std. Err."] = pd.DataFrame(combined_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].sem())
sum_table = sum_table[["Mean Tumor Volume", "Median Tumor Volume", "Tumor Volume Variance", "Tumor Volume Std. Dev.", "Tumor Volume Std. Err."]]

sum_table.head()

,Mean Tumor Volume,Median Tumor Volume,Tumor Volume Variance,Tumor Volume Std. Dev.,Tumor Volume Std. Err.
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
